In [ ]:
#import os ; os.environ['KERAS_BACKEND']='theano'
%run init.ipy
#import logging    
#logging.getLogger('keras').setLevel(logging.INFO)

from utils import *
import miregularizer2
#mi_grad_trainable = False
#sfx = 'v8train'
opts = dict(
    sfx='v8train',
    mi_grad_trainable = True,
    nbepoch = 50,
    batch_size = 128,
    patiencelevel = 50,
    do_pretrain=True,
    N = None,
    EntropyEstimateN = 2000,
    HIDDEN_DIM = 20,
    max_alpha=10.,
)

#sfx = 'v9train'
opts = dict(
    sfx='v9train',
    mi_grad_trainable = True,
    nbepoch = 50,
    batch_size = 128,
    patiencelevel = 50,
    do_pretrain=False,
    N = None,
    EntropyEstimateN = 2000,
    HIDDEN_DIM = 20,
    max_alpha=2.,
)


In [ ]:
#N=10000
d=load_mnist(max_train_items=opts['N'], max_test_items=opts['N']) # , keep_classes = [0,5])

In [ ]:
from keras.layers.normalization import BatchNormalization

model = Sequential()
#model.add(Dense(2*HIDDEN_DIM, input_dim=d.X_train.shape[1], activation='tanh'))
model.add(Dense(opts['HIDDEN_DIM'], input_dim=d.train.X.shape[1], activation='tanh'))
#model.add(BatchNormalization())
#model.add(MILayerTrainable(HIDDEN_DIM, alpha=.1, initlogvar=-1., add_noise=True, entropy_only=False))

import tensorflow as tf

kdelayer, noiselayer, micomputer = None, None, None
kdetraincb, noisetraincb, reportcb = None, None, None

if True:
    kdelayer   = miregularizer2.KDEParamLayer(init_logvar=-5)
    #model.add(BatchNormalization())
    model.add(kdelayer)
    #alltraindata = K.variable(d.train.X)
    regularize_mi_input = d.train.X
    if opts['EntropyEstimateN'] is not None:
        regularize_mi_input = randsample(regularize_mi_input, opts['EntropyEstimateN'])

    #print alltraindata.dtype
    def get_noise_input_func(pd):
        d = tf.constant(pd)
        for layerndx, layer in enumerate(model.layers):
            d = layer(d)
        return d
        
    noiselayer = miregularizer2.GaussianNoise2(init_logvar=-10, kdelayer=kdelayer, 
                                               regularize_mi_input=regularize_mi_input, 
                                               init_alpha=0., 
                                               get_noise_input_func=get_noise_input_func,
                                               trainable=opts['mi_grad_trainable'])
    model.add(noiselayer)

    kdetraincb   = miregularizer2.KDETrain(entropy_train_data=regularize_mi_input, kdelayer=kdelayer)
    if not opts['mi_grad_trainable']:
        noisetraincb = miregularizer2.NoiseTrain(traindata=d.train, noiselayer=noiselayer)
    else:
        noisetraincb = None
    reportcb     = miregularizer2.ReportVars(noiselayer=noiselayer, kdelayer=kdelayer)

    
#model.add(Dense(HIDDEN_DIM, activation='tanh')) # , activity_regularizer=MIRegularizer(0, gaussian_var)))
model.add(Dense(d.train.nb_classes, activation='softmax')) # , activity_regularizer=MIRegularizer(0, gaussian_var)))

def compilefunc():
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    if noiselayer is not None and hasattr(noiselayer, 'logvar'):
        K.set_value(noiselayer.logvar, -10)
    if kdelayer is not None and hasattr(kdelayer, 'logvar'):
        K.set_value(kdelayer.logvar, -10)
    
compilefunc()

In [ ]:
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=opts['patiencelevel'])
#model.fit(X, y, validation_split=0.2, callbacks=[early_stopping])

In [ ]:
#%%timeit -n 1 -r 1
if opts['do_pretrain']:
    if noiselayer is not None:
        K.set_value(noiselayer.mi_regularizer.alpha, 0.0)

    inithist = model.fit(d.train.X, d.train.Y, nb_epoch=opts['nbepoch'], validation_split=0.2,
                     batch_size=opts['batch_size'], verbose=2, # validation_split=0.1, 
                     callbacks=[cb for cb in [kdetraincb,reportcb,early_stopping] if cb is not None])

In [ ]:
fname = "models/initmodel-%s.h5"%opts['sfx']
print "Saving %s"% fname
model.save_weights(fname)
fname = "models/opts-%s.txt"%opts['sfx']
print "Saving %s"% fname
with open(fname, 'w') as f:
    f.write(str(opts))


In [ ]:
#alphavals = np.linspace(0, 5, 20, endpoint=True)
#alphavals = np.exp(np.linspace(0, np.log(6), 20, endpoint=True))-1
#alphavals = np.exp(np.linspace(0, np.log(3.), 30, endpoint=True))-1
#alphavals = np.exp(np.linspace(np.log(5), np.log(10.), 10, endpoint=True))-1
#alphavals = np.exp(np.linspace(0, np.log(10.), 10, endpoint=True))-1
alphavals = np.exp(np.linspace(0, np.log(opts['max_alpha']), 10, endpoint=False))-1

#del saved_hist
#saved_hist = {}
if 'saved_hist' not in locals():
    print "Initializing saved_hist"
    saved_hist = {}
#saved_hist = {}
for alpha in alphavals:
    print "**************** Doing alpha=%.8f, %s ****************" % (alpha, opts['sfx'])
    if alpha in saved_hist:
        continue
    #if alpha < 2 or alpha > 2.2:
    #    continue
    # Reset
    #for clayer in model.layers:
    #    clayer.build(clayer.input_shape)
    
    #if alpha == 0:
    compilefunc()
    model.load_weights("models/initmodel-%s.h5"%opts['sfx'])
        
    if noiselayer is not None :
        K.set_value(noiselayer.mi_regularizer.alpha, alpha)

    hist = model.fit(d.train.X, d.train.Y, nb_epoch=opts['nbepoch'],
                     batch_size=opts['batch_size'], verbose=2, validation_split=0.2, 
                     callbacks=[cb for cb in [kdetraincb, noisetraincb, reportcb, early_stopping] if cb is not None],  # , 
                    )
    saved_hist[alpha] = {'history':hist.history, 
                         'endlogs': miregularizer2.get_logs(model, d, kdelayer, noiselayer, max_entropy_calc_N=opts['EntropyEstimateN'])}
    fname = "models/fitmodel-%s-%0.5f.h5"%(opts['sfx'],alpha)
    print "saving to %s"%fname
    model.save_weights(fname)
    
    if True:
        savedhistfname='models/savedhist-%s.dat'%opts['sfx']
        with open(savedhistfname, 'wb') as f:
            cPickle.dump(saved_hist, f)
        print 'updated', savedhistfname

    print 
    print
raise Exception('Done')

In [ ]:
compilefunc()
#model.load_weights("models/fitmodel-%s.h5"%sfx)
K.set_value(noiselayer.mi_regularizer.alpha, 400.)

hist = model.fit(d.train.X, d.train.Y, nb_epoch=200,
                 batch_size=10., verbose=2, # validation_split=0.1, 
                 callbacks=[kdetraincb, reportcb,]  # noisetraincb, 
                )
print miregularizer2.get_logs(model, d, kdelayer, noiselayer)

In [ ]:
tensor1=noiselayer.get_noise_input_func(tf.constant(d.train.X))
mi_obj_trn = miregularizer2.MIComputer(tensor1, kdelayer=kdelayer, noiselayer=noiselayer)
#print K.eval(mi_obj_trn.get_mi())
#print K.eval(mi_obj_trn.get_hcond())
print K.eval(mi_obj_trn.get_h())
cvar = K.exp(noiselayer.logvar)+K.exp(kdelayer.logvar)

print K.eval(miregularizer2.kde_entropy(tensor1, cvar))

print K.eval(cvar), 'cvar'
cvar2 = K.exp(noiselayer.logvar)
print K.eval(miregularizer2.kde_entropy(noiselayer.get_noise(tensor1), cvar2))
print K.eval(cvar2), 'cvar2'

In [ ]:
print (np.cov(K.eval(tensor1)))
print (np.cov(K.eval(noiselayer.get_noise(tensor1))))


In [ ]:
cinput = K.placeholder()
mi_obj = miregularizer2.MIComputer(cinput, kdelayer=kdelayer, noiselayer=noiselayer)

mifunc = K.function([cinput,],  [mi_obj.get_mi(), mi_obj.get_h(), mi_obj.get_hcond()])
print mifunc([d.train.X,])
cvar = K.exp(noiselayer.logvar)+K.exp(kdelayer.logvar)
print K.eval(cvar)
#        return kde_entropy(self.input, totalvar)
h=K.eval(miregularizer2.kde_entropy(K.variable(activity_trn[fname]), cvar))

In [ ]:
"""
#%%timeit -n 1 -r 1
batch_size = 10
callbacks = []
if noiselayer is not None and kdelayer is not None:
    noiselayer.alpha = 0.0
    callbacks = [miregularizer2.KDETrain(traindata=d, model=model, kdelayer=kdelayer),
                 miregularizer2.NoiseTrain(traindata=d, model=model, noiselayer=noiselayer),
                 miregularizer2.ReportVars(traindata=d, noiselayer=noiselayer,kdelayer=kdelayer),
                ]

hist = model.fit(d.X_train, d.Y_train, nb_epoch=10,
                 batch_size=batch_size, validation_split=0.1, verbose=2, 
                 callbacks=callbacks)
#print hist.history
"""
print

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
#fitmodel1.00000.h5
activity_trn = {}
activity_tst = {}
for fname in ["models/fitmodel-v3-3.64159.h5",]:#"fitmodel.h5", "fitmodel1.00000.h5", "fitmodel2.00000.h5"]:
    compilefunc()
    model.load_weights(fname)

    #activity = get_activations(model, -2, d.X_train)[0]
    #get_activations = K.function([model.layers[0].input, K.learning_phase()], [noiselayer.output,])
    #activations = get_activations([X_batch,0])
    #activity = get_activations([d.X_train, 1])
    #plt.figure()
    #plot_activity(activity, colors=d.y_train, size=1) # , doPCA=False)
    get_activations = K.function([model.layers[0].input, K.learning_phase()], [noiselayer.input,])
    
    activity_trn[fname]=get_activations([d.train.X,])[0]
    activity_tst[fname]=get_activations([d.test.X,])[0]
    

In [ ]:
#print  mi1/np.log(2)
#print  mi2/np.log(2)
cvar= K.exp(noiselayer.logvar)+K.exp(kdelayer.logvar)
cvar2=K.exp(noiselayer.logvar)
cvar = K.variable(np.exp(-10)+np.exp(-8))
print K.eval(noiselayer.logvar)
print K.eval(cvar)
print K.eval(K.exp(noiselayer.logvar))
print K.eval(miregularizer2.kde_entropy(K.variable(activity_trn[fname]), cvar)) # - K.eval(miregularizer2.kde_condentropy(K.variable(activity_trn[fname]), cvar2))

In [ ]:
#print (K.eval(miregularizer2.kde_entropy(K.variable(activity_trn[fname]), cvar)))
print K.eval(miregularizer2.kde_entropy(cvar*K.variable(np.random.randn(*activity_trn[fname].shape)), cvar))
print K.eval(miregularizer2.kde_condentropy(K.variable(np.random.random(activity_trn[fname].shape)), cvar))

In [ ]:
#print np.var(5*np.random.randn(10000))
#print K.eval(miregularizer2.kde_entropy(cvar*K.variable(np.random.randn(*activity_trn[fname].shape)), cvar))
cvar = 0.2
print 'closedform', 0.5*np.log(2*cvar*np.pi*np.e)
print K.eval(miregularizer2.kde_entropy(np.sqrt(cvar)*K.variable(np.random.randn(10000,1)), cvar))

#plt.hist(np.random.randn(10000), bins=100)
False

In [ ]:
fig = plt.figure(figsize=(15,4))
for ndx, fname in enumerate(activity_trn.keys()): # enumerate(["fitmodel.h5", "fitmodel1.00000.h5","fitmodel2.00000.h5"]): #"fitmodel1.50000.h5", 
    for trntstndx in range(2):
        ax = fig.add_subplot(1,2,1+trntstndx) # , projection='3d')
        #plt.subplot(1,4,ndx+1)
        if trntstndx == 0:
            activitydata, titlestr, labels = activity_trn, 'trn', d.train
        else:
            activitydata, titlestr, labels = activity_tst, 'tst', d.test
        if activitydata[fname].shape[1] == 2:
            plt.scatter(activitydata[fname][:,0], activitydata[fname][:,1], c=labels.y, edgecolors='none')
        
        #plot_activity(activitydata[fname], doPCA=True, colors=labels.y, size=2, dims=2, opts=dict(whiten=True))
        plt.title(r'$\alpha = %d$ - (%s)' % (ndx,  titlestr))
        plt.xticks([])
        plt.yticks([])
    
#plt.savefig('imgs/hidden.pdf',bbox_inches='tight')

In [ ]:
print len(d2.X_test)


In [ ]:
plt.figure()
plt.scatter(activity[:,0],activity[:,1])
plt.xlim([-1.5,1.5])
plt.ylim([-1.5,1.5])
plt.figure()
plt.plot(np.var(activity, axis=0))
plt.ylim([0, plt.ylim()[1]])

"""
f1 = K.function([model.layers[0].input, K.learning_phase()], [noiselayer.input])
x1=f1([d.X_train, 0])[0]
from mpl_toolkits.mplot3d import Axes3D
fig=plt.figure(figsize=(10,10))
ax = fig.add_subplot(111, projection='3d')

import sklearn.decomposition

x2 = sklearn.decomposition.PCA(3, whiten=True).fit_transform(x1)
plt.scatter(x2[:,0], x2[:,1], zs=x2[:,2], c=d.y_train, edgecolor='none')
print x2.shape
"""

In [ ]:
#adict = {}
#adict['noreg'] = get_activations(model, 0, X_train)[0]
#adict['entropyonly'] = get_activations(model, 0, X_train)[0]
#adict['mionly'] = get_activations(model, 0, X_train)[0]
#adict['entropynnoise'] = get_activations(model, 0, X_train)[0]
#adict['minnoise'] = get_activations(model, 0, X_train)[0]
#adict['fixednoise'] = get_activations(model, 0, X_train)[0]
#adict['mifixednoise'] = get_activations(model, 0, X_train)[0]


In [ ]:
for k,v in adict.iteritems():
    plot_activity(v)
    plt.title(k)

In [ ]:
Kdists = K.placeholder(ndim=2)
Klogvar = K.placeholder(ndim=0)
from miregularizer2 import kde_entropy_from_dists_loo
N = 1000
dims = 10
lossfunc = K.function([Kdists, Klogvar,], [kde_entropy_from_dists_loo(Kdists, N, dims, K.exp(Klogvar))])


In [ ]:
lossfunc([np.eye(N),1])

In [ ]:
np.array(np.array([1])) # [0]

In [ ]:
fh=K.function([model.layers[0].input,],[noiselayer.mi_obj.get_h()])
f = K.function([model.layers[0].input,],[noiselayer.input])
#.mi_obj.get_h()])

In [ ]:
xx=f([d.X_train,])
print xx
print fh([d.X_train,])

In [ ]:
K.eval(K.exp(noiselayer.logvar)+K.exp(noiselayer.kdelayer.logvar))

In [ ]:
v=K.placeholder(ndim=2)
lv=K.placeholder(ndim=0)
from miregularizer2 import kde_entropy # , get_dims
f2 = K.function([v,lv],[kde_entropy(v,lv)]) # [noiselayer.mi_obj.get_h()])kde_entropy(self.layer.input, totalvar
            

In [ ]:
print xx[0].shape
f2([xx[0],1.])
#import theano
#theano.__version__

In [ ]:
alphavals = np.exp(np.linspace(0, np.log(1.1), 30, endpoint=True))-1
print alphavals

In [ ]:
n = 15
cvar = .5
cov1 = np.eye(n)*cvar
print np.linalg.det(cov1)
print .5**n

print 0.5*n*np.log(2*np.pi*np.e*cvar)
print 0.5*np.log(((2*np.pi*np.e)**n)*np.linalg.det(cov1))
#print 0.5*n*np.log(2*np.pi*np.e*(.5**n))

del n